In [43]:
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
import torch
from torch import nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import DataLoader, TensorDataset,Dataset
import os
import copy

In [44]:
data = np.load('CESM_data_sst_sss_psl_deseason_normalized_resized.npy')
target = np.load('CESM_label_amv_index.npy')

In [152]:
tstep = 86
percent_train = 0.8
ens = 1
channels = 3
batch_size = 32
max_epochs = 10

In [46]:
data.shape

(3, 40, 86, 244, 244)

In [47]:
target.shape

(40, 86)

In [70]:
lead = 10

y = target[:,lead:].reshape(ens*(tstep-lead),1)
X = (data[:,:,:tstep-lead,:,:]).reshape(3,ens*(tstep-lead),244,244).transpose(1,0,2,3)

In [71]:
X_train = torch.from_numpy( X[0:int(np.floor(percent_train*(tstep-lead)*ens)),:,:,:].astype(np.float32) )

X_val = torch.from_numpy( X[int(np.floor(percent_train*(tstep-lead)*ens)):,:,:,:].astype(np.float32) )

y_train = torch.from_numpy(  y[0:int(np.floor(percent_train*(tstep-lead)*ens)),:]  )

y_val = torch.from_numpy( y[int(np.floor(percent_train*(tstep-lead)*ens)):,:]  )

In [64]:
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size)
val_loader   = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size)

In [141]:
model = models.resnet50(pretrained=True)

In [142]:
for param in model.parameters():
    param.requires_grad = False

In [144]:
model.fc = nn.Linear(2048, 1)

In [151]:
opt = torch.optim.Adam(model.parameters())
loss_fn = nn.MSELoss()

In [153]:
epo_train_losses = [loss_fn(model(X_train),y_train)]
epo_val_losses = [loss_fn(model(X_val),y_val)]


for iepoch in tqdm(range(max_epochs)):
    
    batch_train_losses = []
    model.train()
    for x_batch, y_batch in train_loader:
        y_pred = model(x_batch)
        loss = loss_fn(y_pred.squeeze(), y_batch.squeeze())
        batch_train_losses.append(loss.item())
        loss.backward()
        opt.step()
        opt.zero_grad()
    epo_train_losses.append( sum(batch_train_losses)/len(batch_train_losses) )

    batch_val_losses = []
    with torch.set_grad_enabled(False):
        for x_batch_val, y_batch_val in val_loader:
            y_pred = model(x_batch_val)
            loss = loss_fn(y_pred.squeeze(), y_batch_val.squeeze())
            batch_val_losses.append(loss.item())
        epo_val_losses.append( sum(batch_val_losses)/len(batch_val_losses) )

KeyboardInterrupt: 